In [1]:
import numpy as np
import pandas as pd
from functools import partial
from importlib import reload
from collections import Counter
import copy
import matplotlib.pyplot as plt
import testUtils

from TunaSims import tuna_sim
from funcOb import func_ob
import math_distance
import tools

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

Show that Sims are in funtion space

In [2]:
demo_matches = pd.read_pickle('/Users/jonahpoczobutt/projects/TunaRes/metlinGnps_NIST20_matchedPol/intermediateOutputs/splitMatches/train/10_ppm/chunk_1.pkl')
demo_matches_test = pd.read_pickle('/Users/jonahpoczobutt/projects/TunaRes/metlinGnps_NIST20_matchedPol/intermediateOutputs/splitMatches/train/10_ppm/chunk_2.pkl')
demo_query = demo_matches.iloc[0]['query']
demo_target = demo_matches.iloc[0]['target']
demo_query_prec = demo_matches.iloc[0]['precquery']
demo_target_prec = demo_matches.iloc[0]['prectarget']

In [3]:
manhattan_tuna = tuna_sim(demo_query,
              demo_target,
              demo_query_prec,
              demo_target_prec,
              dif_a = 1,
              dif_b = 1,
              unnormed = 1)

dot_tuna = tuna_sim(demo_query,
              demo_target,
              demo_query_prec,
              demo_target_prec,
              mult_a = 1,
              mult_b = 2,
              collapsed = 1,
              mult_norm_a= 1,
              mult_norm_b= 2,
              sim_flip=True)

harmonic_tuna = tuna_sim(demo_query,
              demo_target,
              demo_query_prec,
              demo_target_prec,
              mult_a = 1,
              mult_b = 1,
              expanded = 2,
              add_norm_a= 1,
              add_norm_b= 1,
              sim_flip=True)

demo_query[:,1] /= sum(demo_query[:,1])
demo_target[:,1] /= sum(demo_target[:,1])
combined_old = tools.match_peaks_in_spectra(demo_query, demo_target, ms2_da=0.05)
manhattan = 1 - tools.sigmoid(math_distance.manhattan_distance(combined_old[:,1], combined_old[:,2]))
dot_product = tools.sigmoid(1 - math_distance.dot_product_nosqrt_distance(combined_old[:,1], combined_old[:,2]))
harmonic_mean = tools.sigmoid(1 - math_distance.harmonic_mean_distance(combined_old[:,1], combined_old[:,2]))

print(f'manhattan: {abs(manhattan - manhattan_tuna)}')
print(f'dot_product: {abs(dot_product - dot_tuna)}')
print(f'harmonic_mean: {abs(harmonic_mean - harmonic_tuna)}')

manhattan: 1.9321462207955165e-09
dot_product: 0.0
harmonic_mean: 0.0


Can we recover similarity function from scores and input vectors alone, which training strategies are best?

First We will do no funny biz with cleaning

In [4]:
func_skeletons = [
    func_ob(
    name = "base_10k_iter",
    sim_func = partial(tuna_sim),
    init_vals= 0.1,
    params = None,
    tol = 0,
    lambdas= 1,
    max_iter = 10000,
    epsilon = 1e-5),
    func_ob(
    name = "base_10k_iter_1e2",
    sim_func = partial(tuna_sim),
    init_vals= 0.1,
    params = None,
    tol = 0,
    lambdas= 1,
    max_iter = 10000,
    epsilon = 1e-2),
    func_ob(
    name = "base_10k_iter_zero",
    sim_func = partial(tuna_sim),
    init_vals= 0.1,
    params = None,
    tol = 0,
    lambdas= 1,
    max_iter = 10000,
    epsilon = 1e-2,
    zero_grad_lambda_boost = 2),
    # funcOb.func_ob(
    # name = "base_10k_iter_flex",
    # sim_func = partial(tuna_sim),
    # init_vals= 0.1,
    # params = None,
    # tol = 0,
    # lambdas= 1,
    # max_iter = 10000,
    # epsilon = 1e-2,
    # zero_grad_lambda_boost = 2,
    #),
]

params = {
    "dif_only": ['unnormed','dif_a','dif_b'],
    "dif_and_mult": ['unnormed','dif_a','dif_b','mult_a','mult_b'],
    "collapsed": ['collapsed','dif_a','dif_b','mult_a','mult_b', 'mult_norm_a','mult_norm_b'],
    "expanded": ['expanded','dif_a','dif_b','mult_a','mult_b', 'add_norm_a', 'add_norm_b'],
    "collapsed_and_unnorm": ['unnormed', 'collapsed','dif_a','dif_b','mult_a','mult_b','mult_norm_a','mult_norm_b', 'add_norm_a', 'add_norm_b'],
    "expanded_and_unnorm": ['unnormed', 'expanded','dif_a','dif_b','mult_a','mult_b', 'add_norm_a', 'add_norm_b']  
}

params_dict = dict()
for key, value in params.items():
    params_dict[f'{key}__cleaning'] = value+['query_max_mz_fix',
                                               'target_max_mz_fix', 
                                               'query_fixed_noise', 
                                               'target_fixed_noise',
                                                'query_da_thresh',
                                                'target_da_thresh',
                                                'query_fixed_power',
                                                'target_fixed_power']
    
    params_dict[f'{key}__reweight'] = value+['query_fixed_power',
                                            'query_mz_power',
                                            'query_ent_power',
                                            'target_fixed_power',
                                            'target_mz_power',
                                            'target_ent_power',
                                            'query_reweight_offset',
                                            'target_reweight_offset']
    
    params_dict[f'{key}__cleaning__reweight'] = value+['query_max_mz_fix',
                                               'target_max_mz_fix', 
                                               'query_fixed_noise', 
                                               'target_fixed_noise',
                                                'query_da_thresh',
                                                'target_da_thresh',
                                                'query_fixed_power',
                                                'target_fixed_power',
                                                'query_fixed_power',
                                                'query_mz_power',
                                                'query_ent_power',
                                                'target_fixed_power',
                                                'target_mz_power',
                                                'target_ent_power',
                                                'query_reweight_offset',
                                                'target_reweight_offset']
    
# params.update(new_dict)

Need to turn this into an easy way to generate labels

In [5]:

manhattan_tuna = {'dif_a' : 1,
              'dif_b' : 1,
              'unnormed' : 1}

dot_tuna = {'mult_a' : 1,
              'mult_b' : 2,
              'collapsed' : 1,
              'mult_norm_a': 1,
              'mult_norm_b': 2,
              'sim_flip':True}

harmonic_tuna = {'mult_a' :1,
              'mult_b' : 1,
              'expanded' : 2,
              'add_norm_a' : 1,
              'add_norm_b': 1,
              'sim_flip' : True}

clean1 = {'query_max_mz_fix' : 1.6,
            'query_fixed_noise' : 0.01,
            'query_da_thresh' : 0.05,
            'target_max_mz_fix' : 1.6,
            'target_fixed_noise' : 0.01,
            'target_da_thresh' : 0.05}

clean2 = {'query_max_mz_var' : 0.01,
            'query_var_noise' : 0.01,
            'query_da_thresh' : 0.05,
              'target_max_mz_var': 0.01,
              'target_var_noise' :0.01,
              'target_da_thresh' : 0.05}

clean3 = {'query_max_mz_fix' : 0.8,
        'query_max_mz_var' : 0.005,
        'query_fixed_noise' : 0.01,
        'query_var_noise': 0.01,
        'query_da_thresh' :0.05,
        'target_max_mz_fix' : 0.8,
        'target_max_mz_var' : 0.005,
        'target_fixed_noise' : 0.01,
        'target_var_noise' : 0.01,
        'target_da_thresh' : 0.05}

reweight1 = {'query_fixed_power' : 0.75,
             'target_fixed_power' : 0.75}

reweight2 = {'query_ent_power' : 0.5,
             'target_ent_power' : 0.5}

reweight3 = {'query_ent_power' : 0.1,
              'target_mz_power' :  0.1,
              'query_reweight_offset' : -2,
              'target_reweight_offset' : -2}

reweight4 = {'query_fixed_power' : 0.2,
              'query_mz_power' :0.1,
              'query_ent_power' : 0.25,
              'target_fixed_power' : 0.2,
              'target_mz_power' : 0.1,
              'target_ent_power' : 0.25,
              'query_reweight_offset' : -2,
              'target_reweight_offset' : -2}

sims = {"manhattan": manhattan_tuna,
        "dot_product": dot_tuna,
        "harmoinc": harmonic_tuna}

cleans = {"clean1": clean1,
          "clean2": clean2,
          "clean3": clean3,
          "None": {}}

reweights = {"reweight1":reweight1,
             "reweight2": reweight2,
             "reweight3": reweight3,
             "reweight4": reweight4,
             "None": {}}

funcs_to_find = dict()

for sim, simval in sims.items():
    for clean, cleanval in cleans.items():
        for reweight, reweight_val in reweights.items():
            
            vals = {}
            vals.update(simval)
            vals.update(cleanval)
            vals.update(reweight_val)
    
            funcs_to_find[f'{sim}_{clean}_{reweight}'] = partial(tuna_sim, **vals)

params_to_test = {'manhattan':['dif_only','expanded','dif_and_mult'],
                  'dot': ['collapsed','expanded','collapsed_and_unnorm'],
                  'harmonic': ['collapsed','expanded','collapsed_and_unnorm']
                }

datasets = dict()

demo_matches = pd.read_pickle('/Users/jonahpoczobutt/projects/TunaRes/metlinGnps_NIST20_matchedPol/intermediateOutputs/splitMatches/train/10_ppm/chunk_1.pkl')
demo_matches_test = pd.read_pickle('/Users/jonahpoczobutt/projects/TunaRes/metlinGnps_NIST20_matchedPol/intermediateOutputs/splitMatches/test/10_ppm/chunk_1.pkl')

demo_matches = demo_matches.sample(frac=1)[:100000]
demo_matches_test = demo_matches_test.sample(frac=1)[:100000]

for funcname, func in funcs_to_find.items():

    datasets[funcname] = list()
    datasets[funcname].append(testUtils.create_scores_from_tuna(demo_matches,func))
    datasets[funcname].append(testUtils.create_scores_from_tuna(demo_matches_test,func))

In [ ]:
params_to_test = {'manhattan':['dif_only','expanded','dif_and_mult'],
                  'dot': ['collapsed','expanded','collapsed_and_unnorm'],
                  'harmonic': ['collapsed','expanded','collapsed_and_unnorm']
                }

Train funcs on different scores

In [ ]:
output_file = '/Users/jonahpoczobutt/projects/TunaRes/res_new.csv'
for dataset_name, sets in datasets.items():

    sim_name = dataset_name.split('_')[0]
    params_set = params_to_test[sim_name]

    demo_matches['match'] = sets[0]
    demo_matches_test['match'] = sets[1]

    test_subset = dict()
    for name in params_set:

        for key, value in params_dict.items():

            if name in key:

                test_subset[key] = value
                
    res = testUtils.func_err_tester(func_skeletons, test_subset, {dataset_name: [demo_matches,demo_matches_test]})
    res['sim_name'] = sim_name

    res.to_csv(output_file, mode ='a', header = False)


In [ ]:
sets

In [ ]:
params_dict